In [21]:
try:
    %tensorflow_version 2.x
except Exception:
    pass

In [22]:
from __future__ import absolute_import, division, print_function, unicode_literals, unicode_literals

# tensorflow와 tf.keras를 import
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import random

print(tf.__version__)

2.2.0


In [23]:
(x_train_all, y_train_all), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [24]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_all, y_train_all, stratify=y_train_all, test_size=0.2, random_state=42)
x_train = x_train / 255.0
x_val = x_val / 255.0

In [25]:
print("x_train:", x_train.shape)
print("y_train:", y_train.shape)
print("x_val:", x_val.shape)
print("y_val:", y_val.shape)

x_train: (48000, 28, 28)
y_train: (48000,)
x_val: (12000, 28, 28)
y_val: (12000,)


In [26]:
from tensorflow.keras.utils import to_categorical
y_train_encoded = tf.keras.utils.to_categorical(y_train)
y_val_encoded = tf.keras.utils.to_categorical(y_val)

In [27]:
x_train = x_train.reshape(-1, x_train.shape[1], x_train.shape[2], 1)
x_val   = x_val.reshape(-1, x_val.shape[1], x_val.shape[2], 1)

In [28]:
print("x_train:", x_train.shape)
print("y_train:", y_train_encoded.shape)
print("x_val:", x_val.shape)
print("y_val:", y_val_encoded.shape)

x_train: (48000, 28, 28, 1)
y_train: (48000, 10)
x_val: (12000, 28, 28, 1)
y_val: (12000, 10)


In [29]:
print(np.min(x_train), np.max(x_train))
x_train = x_train - np.min(x_train)

print(np.min(x_train), np.max(x_train))
x_train = x_train / np.max(x_train)

print(np.min(x_train), np.max(x_train))

print(np.min(x_val), np.max(x_val))
x_val = x_val - np.min(x_val)

print(np.min(x_val), np.max(x_val))
x_val = x_val / np.max(x_val)

print(np.min(x_val), np.max(x_val))

0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0
0.0 1.0


In [30]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.layers import Dropout

In [31]:
from tensorflow.keras.layers import Dropout

conv2 = tf.keras.Sequential()

conv2.add(Conv2D(32, (3, 3), activation="relu", padding="same", input_shape=(x_train.shape[1], x_train.shape[2], 1)))
# conv2.add(MaxPooling2D((2, 2)))
conv2.add(Conv2D(64, (3, 3), activation="relu", padding="same"))
conv2.add(MaxPooling2D((2, 2)))
conv2.add(Dropout(0.5))

conv2.add(Flatten())
conv2.add(Dense(128, activation="relu"))
conv2.add(Dropout(0.25))
Dense

conv2.add(Dense(y_train_encoded.shape[1], activation="softmax"))

In [32]:
conv2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 28, 28, 32)        320       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               1605760   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)              

In [33]:


conv2.compile(optimizer=keras.optimizers.SGD(lr=0.01, momentum=0.0, decay=0.0, nesterov=False), loss=tf.keras.losses.categorical_crossentropy, metrics=["accuracy"])

In [ ]:
history = conv2.fit(x_train, y_train_encoded, batch_size=4, epochs=20, verbose=1, validation_data=(x_val, y_val_encoded))

Epoch 1/20
12000/12000 [==============================] - 68s 6ms/step - loss: 0.5384 - accuracy: 0.8033 - val_loss: 0.3687 - val_accuracy: 0.8574
Epoch 2/20
12000/12000 [==============================] - 67s 6ms/step - loss: 0.3620 - accuracy: 0.8680 - val_loss: 0.2935 - val_accuracy: 0.8914
Epoch 3/20
11072/12000 [==========================>...] - ETA: 4s - loss: 0.3117 - accuracy: 0.8850

In [ ]:
plt.figure(figsize=(14, 6))
plt.rc('font', size=18)

plt.plot(history.history["accuracy"])
plt.plot(history.history["val_accuracy"])
plt.ylabel("accuracy")
plt.xlabel("epoch")
plt.legend(["train_accuracy", "val_accuracy"])
plt.show()